# Проект SF Scoring - построение модели банковского скоринга.
Автор - Пальванова Юлия. Версия 1, от 3 марта 2021

# 0. Введение.
В этом проекте рассматривается задача банковского скоринга - принятие решения о том, будет ли заёмщик проблемным (т.е. допустит дефолт по кредиту) или нет. Решение принимается по набору различных признаков для каждого клиента, таких как наличие машины, загранпаспорта, доходы, и прочие. Для принятия решений о новых клиентах используется модель, обученная на исторических данных о нескольких десятках тысяч клиентов, о которых известно, были ли они проблемными или нет. Таким образом, задача является бинарной классификацией, и может быть решена различными методами: логистической регрессией, методом опорных векторов, различными методами решающих деревьев, и простыми методами классификации (наивный байесовский классификатор и kNN). Основной метрикой соревнования на kaggle является ROC AUC Score, хотя также будут проанализированы и другие метрики классификации (confusion matrix, accuracy, precision, recall, F1 score).

# Cодержание.
1) Анализ данных.

2) Отбор и трансформация признаков

3) Применение простейших моделей - наивный Байес и kNN, валидация и подбор гиперпараметров.

4) Применение линейных моделей логистической регрессии и метода опорных векторов, валидация, подбор гиперпараметров.

5) Применение XGBoost, валидация и подбор гиперпараметров.

6) Сравнение метрик, выводы.

# 1. EDA.
Датасет уже очень хорошо подготовлен. Для упрощения EDA будем использовать модуль dataprep.eda.

In [ ]:
 This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#!pip install -U dataprep
!pip install xgboost
!pip install catboost
#import dataprep.eda
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.pipeline import make_pipeline, Pipeline, FeatureUnion
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler, FunctionTransformer, MinMaxScaler, PolynomialFeatures
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate, StratifiedShuffleSplit
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.feature_selection import f_classif,mutual_info_classif, SelectKBest, SelectFromModel
from sklearn.metrics import (plot_confusion_matrix, plot_roc_curve, f1_score, confusion_matrix, 
                             precision_score, precision_recall_curve, accuracy_score, recall_score, 
                             plot_precision_recall_curve, roc_auc_score)
from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn import set_config
import xgboost as xgb
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

set_config(display='diagram')


from pandas import Series
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression


from sklearn.metrics import confusion_matrix
from sklearn.metrics import auc, roc_auc_score, roc_curve

import warnings
from dateutil import parser
import dateutil
from datetime import datetime, timedelta
import datetime
import plotly as px
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf

from statistics import variance
# импорт дробей как значений параметров
from fractions import Fraction as fr
import statistics
import matplotlib.pyplot as plt
%matplotlib inline
warnings.simplefilter('ignore')

pd.set_option('display.max_rows', 50)  # показывать больше строк
pd.set_option('display.max_columns', 50)  # показывать больше колонок
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import xml.etree.ElementTree as ET 
#!pip install xmljson
#import xmljson
import pickle  
from datetime import datetime  
from os import path  
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error
import json  
from pprint import pprint  
pd.set_option('display.max_columns', None)

In [ ]:
#подгрузка данных
train = pd.read_csv('train.csv')
test= pd.read_csv('test.csv')
sample_sub = pd.read_csv('sample_submission.csv')


# 1.Анализ данных.

In [ ]:
train.info()

In [ ]:
train.describe()

In [ ]:
test.info()

In [ ]:
test.describe()

In [ ]:
train.columns, test.columns

### Описания полей - копипаста предоставленного описания.

- client_id - идентификатор клиента
- education - уровень образования
- sex - пол заемщика
- age - возраст заемщика
- car - флаг наличия автомобиля
- car_type - флаг автомобиля иномарки
- mdecline_app_cnt - количество отказанных прошлых заявок
- good_work - флаг наличия “хорошей” работы
- bki_request_cnt - количество запросов в БКИ
- home_address - категоризатор домашнего адреса
- work_address - категоризатор рабочего адреса
- income - доход заемщика
- foreign_passport - наличие загранпаспорта
- sna - связь заемщика с клиентами банка
- first_time - давность наличия информации о заемщике
- score_bki - скоринговый балл по данным из БКИ
- region_rating - рейтинг региона
- app_date - дата подачи заявки
- default - флаг дефолта по кредиту

Первым делом посмотрим пропуски.

In [ ]:
train.isnull().sum(), test.isnull().sum()

Для наглядности посмотрим признак education на графике

In [ ]:
train['education'].value_counts().plot.barh()

In [ ]:
test['education'].value_counts().plot.barh()

Пропущенные значения есть только в столбце education. Заполним эти пропуски модой. Для того, чтобы избежать даже минимальной утечки данных из трейна в тест, заполнение пропусков в тесте надо проводить тем же значением, что и в трейне (то есть не считать моду по тесту, а взять её из трейна). И вообще, все шаги подготовки данных производятся на трейне, а применяются потом к тесту\валидации. Поэтому будем готовить pipeline, чтобы гарантировать отсутствие утечек данных. Для подготовки столбцов, которые у нас весьма разнородные, будем использовать ColumnTransfromer.



In [ ]:
mport collections

c_1 = collections.Counter(train['education'])
c_2 = collections.Counter(test['education'])

train['education'].fillna(c_1.most_common()[0][0], inplace=True)
test['education'].fillna(c_2.most_common()[0][0], inplace=True)


Проверим успешность заполнения

In [ ]:
train.isnull().sum(), test.isnull().sum()

In [ ]:
train['education'].value_counts().plot.barh()

In [ ]:
test['education'].value_counts().plot.barh()

Для удобного анализа сгруппируем признаки в три категории по типу их обработки (категориальные, бинарные и числовые)

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
# бинарные переменные
ord_cols=['region_rating', 'first_time']
# котегориальные переменные
cat_cols=['sex', 'car', 'car_type','foreign_passport', 'work_address', 'home_address', 'sna', 'education']
# числовые переменные
num_cols=['decline_app_cnt', 'bki_request_cnt', 'income', 'age']

Посмотрим на распределение числовых данных:

In [ ]:
train.hist()

In [ ]:
train.describe()

Построим графики распределения логарифмированных переменных.

In [ ]:
for i in num_cols:
    plt.figure()
    sns.distplot(train[i][train[i] > 0].dropna(), kde = False, rug=False)
    plt.title(i)
    plt.show()

Все признаки имеют смещение к левому краю.

Построим boxplot’ы для численных переменных и посмотрим их распределение по отношение к целевой переменной default

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
!pip install plotnine

In [ ]:
from plotnine import ggplot, aes, geom_boxplot

### Возраст

In [ ]:
sns.catplot(x='default',
            y='age',
            kind='boxen',
            data=train);

Количество отказанных прошлых заявок¶

In [ ]:
sns.catplot(x='default',
            y='decline_app_cnt',
            kind='boxen',
            data=train);

### Количество запросов БКИ

In [ ]:
sns.catplot(x='default',
            y='bki_request_cnt',
            kind='boxen',
            data=train);

### Доход

In [ ]:
sns.catplot(x='default',
            y='income',
            kind='boxen',
            data=train);

### Вывод по количественным признакам.
Выбросы есть во всех количественных признаках. Но они не кажутся неадекватными, и их доля очень невелика. Поэтому исправлять мы их не будем.

### Корреляция¶
Посмотрим на корреляцию признаков в тестовой и основной выборках

In [ ]:
sns.set(font_scale=1)
plt.subplots(figsize=(12, 12))
sns.heatmap(test.corr(), square=True,
              annot=True, fmt=".1f", linewidths=0.1, cmap="RdBu")

Смотрим, у каких пар признаков сильная взаимосвязь:

Домашний адрес и адрес работы. Довольно логично, так как в основном люди живут и работают в одном населенном пункте.
Связь заемщика с клиентами банка и давность наличия информации о заемщике. Тоже в полеъне логично
Теперь посмотрим кореляцию признаков на файле трейн

In [ ]:
sns.set(font_scale=1)
plt.subplots(figsize=(12, 12))
sns.heatmap(train.corr(), square=True,
              annot=True, fmt=".1f", linewidths=0.1, cmap="RdBu")

Картина с корреляцией признаков аналогичная. При этом, большой корреляции с целевой переменной нет ни у дного признака. Максимальная корреляция с коэффициентом 0.2 наблюдается только целевой переменной со скоринговым баллом по данным из БКИ

С моделью в общем все ясно. Теперь поработаем над данными.

# 2. Отбор и трансформация признаков


In [ ]:
#загоним app_date в формат unix timestamp
train_for_selection = train.drop(columns = ['client_id', 'default'])
train_for_selection['app_date'] = train_for_selection['app_date'].apply(lambda x: pd.to_datetime(x).timestamp())
train_for_selection['education'].fillna(train_for_selection.education.mode(), inplace = True)
train_for_selection = pd.get_dummies(train_for_selection, columns=cat_cols+ord_cols)

train_for_selection.sample(5)

### ANOVA

In [ ]:
plt.figure(figsize=(15,15))
pd.Series(
    dict(zip(train_for_selection.columns, 
             f_classif(train_for_selection, train['default'])[0])))\
    .sort_values().plot(kind = 'barh')

### Mutual inforamtion.

In [ ]:
plt.figure(figsize=(15,15))
pd.Series(dict(
    zip(
        train_for_selection.columns, 
                   mutual_info_classif(train_for_selection, train['default']))))\
                   .sort_values().plot(kind='barh')

### Вывод.
Мало влияющие на результат признаки есть. Для линейных методов цели снизить размерность пространства категориальных признаков не стоит, потому что и так всё работает быстро. Для SVM при применении kernel trick вычисления становятся довольно длительными, поэтому будем пробовать отбирать признаки.

Итого, вот что мы делаем с признаками в "базовом" варианте:
- client_id - идентификатор клиента. Для модели он не нужен. Трогать его не будем, понадобится исключительно для соотнесения предикта с истинным значением на тесте и валидации.
- education - уровень образования, категориальный признак, применим к нему OneHotEncoder.
- sex - пол заемщика - бинарный категориальный признак, применим к нему OneHotEncoder, который в данном случае выдаст ровно один признак.
- age - возраст заемщика - количественный признак. Правый хвост довольно тяжёлый, распределение похоже на логнормальное. Возьмём логарифм, применим StandardScaler.
- car - флаг наличия автомобиля - категоральный бинарный, OneHotEncoder
- car_type - флаг автомобиля иномарки - категоральный бинарный, OneHotEncoder
- decline_app_cnt - количество отказанных прошлых заявок - количественный признак, больше всего вообще напоминает экспоненциальное распределение. Прибавим единицу и возьмём логарифм.
- good_work - флаг наличия “хорошей” работы - бинарный, можно вообще не трогать
- bki_request_cnt - количество запросов в БКИ - аналогично decline_app_cnt
- home_address - категоризатор домашнего адреса. Вообще не до конца понятно, что это значит. Тут 3 категории (не понятно о чем каждая из категорий говорит). Непонятно, есть ли тут отношение порядка. Попробуем просто перекодировать в dummies.
- work_address - категоризатор рабочего адреса - аналогично предыдущему
- income - доход заемщика - количественный признак, с тяжёлым хвостом. Логарифм, StandardScaler
- foreign_passport - наличие загранпаспорта, бинарный категориальный признак, onehotencoder
- sna - связь заемщика с клиентами банка, абсолютно непонятно значение этого признака. Будем пока считать категориальным и применим onehotencoder
- first_time - давность наличия информации о заемщике. Природа её, возможно и числовая, но тут всего 4 значения. Опять же напрашивается onehotencoder.
- score_bki - скоринговый балл по данным из БКИ, количественная, кравсивое нормальное распределение, применим standard scaler.
- region_rating - рейтинг региона. Опять же, имеет числовую природу, но всего 4 значения - растаскиваем на dummies.
- app_date - дата подачи заявки - сама по себе бесполезна, но можно повытягивать дополнительные признаки.
- default - флаг дефолта по кредиту - целевая переменная, не трогаем.

Попробуем получить дополнительные признаки из даты

In [ ]:
train_for_features = train.copy()
train_for_features['app_date'] = train_for_features['app_date'].apply(pd.to_datetime)
print(train_for_features['app_date'].dt.year.nunique())

Год сразу выбрасываем. Возможные признаки: месяц, число, день недели, выходной или нет. При этом маловероятно, что голое число хоть с чем-то будет коррелировать.

In [ ]:
train_for_features['month'] = train_for_features['app_date'].dt.month
train_for_features['day'] = train_for_features['app_date'].dt.day
train_for_features['weekday'] = train_for_features['app_date'].dt.weekday
train_for_features['is_weekend'] = train_for_features['weekday'].apply(lambda x: 1 if x in (5,6) else 0)

train_for_features.sample(5)

In [ ]:
sns.set(font_scale=1)
plt.subplots(figsize=(12, 12))
sns.heatmap(train_for_features.corr(), square=True,
              annot=True, fmt=".1f", linewidths=0.1, cmap="RdBu")

Полученные признаки никак не коррелируют с целевой переменной. Зато, очевидно, день недели коррелирует с признаком "выходной". Не будем пользоваться последним признаком. Лучше потом weekday растащим на dummy-переменные. И признак "число" выкинем, он кажется бесполезным. Напишем класс для генерации новых признаков, со стандартным интерфейсом sklearn, чтобы потом его можно было встроить в модель.

In [ ]:
class DateFeatureGenerator(BaseEstimator, TransformerMixin):
    def __init__(self, date_col):
        self.date_col = date_col
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        dates = pd.to_datetime(X[self.date_col])
        ts = dates.apply(lambda x: x.timestamp()).values.reshape(-1,1)
        weekdays = dates.dt.weekday.values.reshape(-1,1)
        months = dates.dt.month.values.reshape(-1,1)
        scl = MinMaxScaler()
        ohe = OneHotEncoder(sparse = False)
        return np.hstack((ohe.fit_transform(weekdays), scl.fit_transform(ts), ohe.fit_transform(months)))

Распределим признаки по категориям, и напишем код для обработки образования - onehotencoder и числовых переменных, требующих логарифмирования и нормализации.

In [ ]:
cat_features = ['sex', 'car', 'car_type','foreign_passport', 'work_address', 'home_address', 'sna']
num_features = ['decline_app_cnt', 'bki_request_cnt', 'income', 'age']
ord_features = ['region_rating', 'first_time']
features_to_leave_untouched = ['good_work']
edu_transform_pipe = OneHotEncoder(sparse=False) # код для обработки образования
#make_pipeline(SimpleImputer(strategy='most_frequent'), 
numeric_transformer = make_pipeline(FunctionTransformer(func = np.log1p), StandardScaler()) #код для всех количсественных
remaining_cols = set(train.columns) - set(cat_features)- set(num_features)- set(ord_features) - set(['education']) - set(features_to_leave_untouched)
print(remaining_cols)

Теперь напишем самый главный код по обработке всеx признаков

In [ ]:
features_encoder = make_column_transformer(
    (FunctionTransformer(func = lambda x: x), features_to_leave_untouched),  #костыль, чтобы не мучиться с remainder и не вытаскивать из него app_date
    (edu_transform_pipe, ['education']),
    (OneHotEncoder(drop='if_binary', sparse=False), cat_features+ord_features),
)
num_transformer = make_column_transformer(
    (numeric_transformer, num_features),
    (StandardScaler(), ['score_bki'])
)
poly_num_transformer = make_pipeline(num_transformer, PolynomialFeatures(degree=2))

# Делаем матрицу плана
total_features = FeatureUnion([('gen', DateFeatureGenerator(date_col = 'app_date')),
                               ('encode', features_encoder),
                               ('trans_num', num_transformer)
                                ])
total_features.fit(train.drop(columns = ['default', 'client_id']))
train_features = total_features.transform(train.drop(columns = ['default', 'client_id']))
test_features = total_features.transform(test.drop(columns = ['client_id']))


poly_total_features = FeatureUnion([('gen', DateFeatureGenerator(date_col = 'app_date')),
                               ('encode', features_encoder),
                               ('trans_num_poly', poly_num_transformer)
                                ])
poly_total_features.fit(train.drop(columns = ['default', 'client_id']))
poly_train_features = poly_total_features.transform(train.drop(columns = ['default', 'client_id']))
poly_test_features = poly_total_features.transform(test.drop(columns = ['client_id']))

К сожалению, при обработке не удалось сохранить имена столбцов (потому что на выходе получаем массив numpy). Будем работать с чем есть

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_features, train['default'], random_state=42, test_size=0.2)
X_train_p, X_test_p, y_train_p, y_test_p = train_test_split(poly_train_features, train['default'], random_state=42, test_size=0.2)

# 3.Простые модели.
## 3.1. Naive Bayes.
При применении наивного байесовского классификатора сразу нужно держать в голове, что его "наивное" предположение о независимости признаков у нас, скорее всего не выполнится. Поэтому сначала попробуем запустить его на полной матрице плана, а потом на топ-10, 20, 30 и 40 признаках по ANOVA, как раз отсечём скоррелированные друг с другом признаки.

In [ ]:
def construct_metrics(name, clf, X, y_true):
    y_pred = clf.predict(X)
    y_proba = clf.predict_proba(X)[:,1]
    results = pd.DataFrame(
         {
            'Classifier': str(clf),
            'Accuracy': accuracy_score(y_true, y_pred),
            'Precision': precision_score(y_true, y_pred),
            'Recall': recall_score(y_true, y_pred),
            'F1': f1_score(y_true, y_pred),
            'ROC AUC': roc_auc_score(y_true, y_proba)
         }, index = [name])
    plot_confusion_matrix(clf, X, y_true.values.reshape(-1,1))
    plot_roc_curve(clf, X, y_true.values.reshape(-1,1))
    plot_precision_recall_curve(clf, X, y_true.values.reshape(-1,1))
    return results
results = []


In [ ]:
gnb = GaussianNB()
gnb.fit(X_train, y_train)
results = construct_metrics('Gauss NB without feature selection', gnb, X_test, y_test)
results

Получилось что получилось. Мы ввели второе допущение по поводу нормального распределения признаков. Получили много ошибок, (первого рода, если нулевая гипотеза у нас о том, что клиент дефолтный), или False-Positive(если считать positive = клиент недефолтный). Посмотрим что с отсечением признаков.

In [ ]:
nb_with_selection = make_pipeline(SelectKBest(), GaussianNB())
param_grid = {
    'selectkbest__k':[10,20,30,37]
    }
gridsearch = GridSearchCV(nb_with_selection, param_grid, scoring='f1', n_jobs=-1)

In [ ]:
gridsearch.fit(X_train, y_train)
model = gridsearch.best_estimator_
print(model)

In [ ]:
results = results.append(construct_metrics('Gauss NB with 36 features selected', model, X_test, y_test))
results.iloc[-1]

Ничего не поменялось

In [ ]:
gnb = GaussianNB()
gnb.fit(X_train_p, y_train_p)
results = results.append(construct_metrics('Gauss NB poly_features', gnb, X_test_p, y_test_p))
results.iloc[-1]

Полиномиальные признаки так же рне дали хорошего результата

## 3.2. K ближайших соседей.

In [ ]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
results = results.append(construct_metrics('KNN, default parameters', knn, X_test, y_test))
results.iloc[-1]

Дно, попробуем оптимизировать гиперпараметры. А с полиномиальными признакими ничего не будем делать

In [ ]:
param_grid = [
    {'n_neighbors': range(1,6,1), 
     'weights': ['uniform', 'distance'], 
     'p':[1, 2]
    }
] # остальные гиперпараметры не до конца понятны, а расчёты занимают какое-то время, поэтому лезть туда не стану
gridsearch = GridSearchCV(knn, param_grid, scoring='f1', n_jobs=-1)
gridsearch.fit(X_train, y_train)
model = gridsearch.best_estimator_
model.get_params()

In [ ]:
results = results.append(construct_metrics('KNN, 1 nearest neighbor', model, X_test, y_test))
results.iloc[-1]

KNN сработал слабо. С учётом большого количества категориальных признаков это неудивительно.

# 4. Логистическая регрессия.

In [ ]:
# сначала параметры по умолчанию, только дадим побольше итераций.
logreg = LogisticRegression(max_iter = 10000, random_state=42)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
results = results.append(construct_metrics('Logistic regression, default parameters', logreg, X_test, y_test))
results.iloc[-1]

ROC AUC-удовлетворительные, а вот с recall проблема, он вообще никакой. Если оторваться от целевой метрики ROC AUC, то это значит что мы очень плохо отсеиваем дефолтных клиентов. А всё потому, что классы несбалансированы - недефолтных клиентов гораздо больше, чем дефолтных. Поэтому все дальнейшие модели будем запускать с class_weight='balanced'.

In [ ]:
logreg.coef_

Веса в регрессии не взорвались, всё в порядке. Видно, что какие-то признаки получают довольно большие коэффициенты, какие-то - близкие к нулю.

Попробуем сбалансировать классы.

In [ ]:
logreg = LogisticRegression(max_iter = 10000, random_state=42, class_weight='balanced')
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
results = results.append(construct_metrics('Logistic regression, default parameters, balanced classes', logreg, X_test, y_test))
results.iloc[-1]

Так recall у нас улучшился, но зато теперь мы допускаем больше ошибок второго рода (считаем недефолтных клиентов дефолтными). Попробуем полиномиальные признаки.

In [ ]:
logreg = LogisticRegression(max_iter = 10000, random_state=42,class_weight='balanced')
logreg.fit(X_train_p, y_train_p)
y_pred = logreg.predict(X_test_p)
results = results.append(construct_metrics('Logistic regression, default parameters w\poly features', logreg, X_test_p, y_test_p))
results.iloc[-1]

Огромных качественных изменений не получилось. Попробуем оптимизировать параметры на оба случая.

In [ ]:
logreg = LogisticRegressionCV(
    Cs = np.linspace(0.2,2,10),
    penalty = 'l2',
    scoring = 'f1',
    max_iter = 10000, 
    random_state=42, 
    class_weight='balanced')
logreg.fit(X_train, y_train)
results = results.append(construct_metrics('Logistic regression, optimized C for L2',logreg, X_test, y_test))
results.iloc[-1]

In [ ]:
logreg = LogisticRegressionCV(
    Cs = np.linspace(0.2,2,10),
    penalty = 'l1',
    scoring = 'f1',
    solver = 'saga',
    max_iter = 10000, 
    random_state=42, 
    class_weight='balanced')
logreg.fit(X_train, y_train)
results = results.append(construct_metrics('Logistic regression, optimized C for L1',logreg, X_test, y_test))
results.iloc[-1]

In [ ]:
logreg_elastic = LogisticRegressionCV(
    Cs = np.linspace(0.2,2,10),
    penalty = 'elasticnet',
    l1_ratios = np.linspace(0,1,6),
    scoring = 'f1',
    solver = 'saga',
    max_iter = 10000, 
    random_state=42, 
    class_weight='balanced')
logreg_elastic.fit(X_train, y_train)
results = results.append(construct_metrics('Logistic regression, optimized C and L1_ratio for elasticnet',logreg_elastic, X_test, y_test))
results.iloc[-1]

Регуляризация по L1 и L2 нормам выдаёт результат незначительно лучше.

Посмотрим полиномиальные признаки для регуляризации elasticnet.

In [ ]:
logreg = LogisticRegressionCV(
    Cs = np.linspace(0.4,1,4),
    penalty = 'elasticnet',
    l1_ratios = np.linspace(0,1,4),
    scoring = 'f1',
    solver = 'saga',
    max_iter=10000, 
    tol = 0.001, #чтобы быстрее считалось
    random_state=42, 
    class_weight='balanced')
logreg.fit(X_train_p, y_train_p)
results = results.append(construct_metrics('Logreg with poly features optimized', logreg, X_test_p, y_test_p))
results.iloc[-1]

Итого, лучший результат по ROC AUC получается при использовании регуляризации elasticnet без полиномиальных признаков. Параметры регуляризации такие:

In [ ]:
print(f'C = {logreg_elastic.C_[0]}, l1_ratio = {logreg_elastic.l1_ratio_[0]}')

Проведём кросс-валидацию.

In [ ]:
best_logreg = LogisticRegression(
        C = 0.2,
        penalty = 'elasticnet',
        l1_ratio = 0.8,
        solver = 'saga',
        max_iter=10000, 
        tol = 0.001, #чтобы быстрее считалось
        random_state=42, 
        class_weight='balanced')
cvsplitter = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
cross_validate(best_logreg, X_train, y_train, 
               scoring=('recall', 'f1', 'roc_auc'), cv = cvsplitter)

Модель немного переобучается - метрики на кросс-валидации похуже.

Обучим модель на полном трейне и сделаем файл для сабмита.

In [ ]:
best_logreg.fit(train_features,train['default'].values)

In [ ]:
pd.DataFrame({'client_id':test.client_id.values, 'default':best_logreg.predict_proba(test_features)[:,1]}).to_csv('submission_best_logreg.csv', index = False)

# 5. Метод опорных векторов.
Начнём с линейного SVC. В SVC нет метода predict_proba (потому что там нет вероятностей, есть только margin), вместо него decision function, поэтому перепишем метод для метрик.

In [ ]:
def construct_metrics_svc(name, clf, X, y_true):
    y_pred = clf.predict(X)
    y_proba = clf.decision_function(X)
    results = pd.DataFrame({
            'Classifier': str(clf),
            'Accuracy': accuracy_score(y_true, y_pred),
            'Precision': precision_score(y_true, y_pred),
            'Recall': recall_score(y_true, y_pred),
            'F1': f1_score(y_true, y_pred),
            'ROC AUC': roc_auc_score(y_true, y_proba)
         }, index = [name])
    plot_confusion_matrix(clf, X, y_true.values.reshape(-1,1))
    plot_roc_curve(clf, X, y_true.values.reshape(-1,1))
    plot_precision_recall_curve(clf, X, y_true.values.reshape(-1,1))
    return results

In [ ]:
svmclf = LinearSVC(class_weight='balanced', max_iter=10000)
svmclf.fit(X_train, y_train)

In [ ]:
results = results.append(construct_metrics_svc('LinearSVC',svmclf, X_test, y_test))
results.iloc[-1]

Кросс-валидация и подбор гиперпараметров:

In [ ]:
params = {
    'C': np.linspace(0.2,2,5),
    'max_iter':[10000]
}
cv = GridSearchCV(svmclf, params, scoring = 'f1', cv=5, n_jobs = 8)
cv.fit(X_train, y_train)

In [ ]:
results = results.append(construct_metrics_svc('Linear SVC optimized',cv, X_test, y_test))
results.iloc[-1]

Попробуем kernel trick, со стандартным ядром radial basis function

In [ ]:
kernel_svc = SVC(class_weight='balanced', max_iter=-1)
kernel_svc.fit(X_train, y_train)
results = results.append(construct_metrics_svc('SVC with kernel trick', kernel_svc, X_test, y_test))
results.iloc[-1]

Очень медленно. Боюсь, что не до кросс-валидации. Обучим на всём трейне линейный SVC и сделаем файл для сабмита.

In [ ]:
cv.fit(train_features,train['default'].values)
pd.DataFrame({'client_id':test.client_id.values, 
              'default':cv.decision_function(test_features)})\
              .to_csv('submission_best_svm.csv', index = False)

SVC показал себя не лучше логрегрессии, но работает медленно.

Говорят, что решающие деревья хорошо подходят для подобных задач.

Попробуем их, но не просто в виде решающих деревьев, а с бустингом.

# 6. XGBoost и Catboost
## 6.1. XGBoost.


In [ ]:
balance_factor = (len(y_train) - sum(y_train))/(sum(y_train)) #пока сбалансируем классы так
xg = xgb.XGBClassifier(max_depth = 3, max_bin = 1000, scale_pos_weight = balance_factor, eval_metric = 'auc')
xg.fit(X_train, y_train)
results = results.append(construct_metrics('XG Boost', xg, X_test, y_test))
results.iloc[-1]

ROC AUC получилась относительно хорошая, f1 - примерно как раньше. Попробуем подобрать максимальное количество деревьев.

In [ ]:
matr = xgb.DMatrix(X_train, label=y_train)
xg = xgb.XGBClassifier(missing=9999999999,
                    max_depth = 7,
                    n_estimators=700,
                    learning_rate=0.1, 
                    nthread=8,
                    subsample=1.0,
                    colsample_bytree=0.5,
                    min_child_weight = 3,
                    seed=42,
                    scale_pos_weight = balance_factor, 
                    eval_metric = 'auc')
xgb_param = xg.get_xgb_params()
cvresult = xgb.cv(xgb_param, matr, num_boost_round=5000, nfold=15, metrics=['auc'],
     early_stopping_rounds=50, stratified=True, seed=42)
print('Best number of trees = {}'.format(cvresult.shape[0]))

xg.set_params(n_estimators=cvresult.shape[0])

xg.fit(X_train, y_train)
results = results.append(construct_metrics('XG Boost with optimiezed number of trees', xg, X_test, y_test))
results.iloc[-1]

Прекрасно, пока что лучший результат. Обучим xgboost на всем доступном трейне.

In [ ]:
matr = xgb.DMatrix(train_features, label=train['default'].values)
xg.fit(train_features,train['default'].values)
pd.DataFrame({'client_id':test.client_id.values, 
              'default':xg.predict_proba(test_features)[:,1]})\
              .to_csv('submission_xgboost.csv', index = False)

## 6.2. Catboost.

In [ ]:
from catboost import CatBoostClassifier, cv, Pool
balance_factor = (len(y_train) - sum(y_train))/(sum(y_train))
catboost = CatBoostClassifier(task_type = 'CPU', 
                              verbose = False, 
                              class_weights = [1, balance_factor])
catboost.fit(X_train, y_train)
results = results.append(construct_metrics('catBoost default', catboost, X_test, y_test))
results.iloc[-1]

Оптимизируем гиперапараметры.

In [ ]:
catboost = CatBoostClassifier(task_type = 'CPU', 
                              verbose = False, 
                              class_weights = [1, balance_factor])
grid = {'learning_rate': [0.03, 0.1],
        'depth': [4, 6, 10],
        'l2_leaf_reg': [1, 3, 5, 7, 9]}

grid_search_result = catboost.grid_search(grid, 
                                       X=X_train, 
                                       y=y_train, 
                                       plot=True)

In [ ]:
results = results.append(construct_metrics('catBoost depth cv optimized', catboost, X_test, y_test))
results.iloc[-1]

Примем это за финальный результат.

In [ ]:
catboost.fit(train_features,train['default'].values)
pd.DataFrame({'client_id':test.client_id.values, 'default':catboost.predict_proba(test_features)[:,1]}).to_csv('submission_catboost.csv', index = False)

# 7. Выводы.
Из рассмотренных методов классификации наименее эффективными оказались методы наивного байесовского классификатора и kNN.
Эффективность логистичесокй регрессии и методов, основанных на деревьях и бустинге, в данной задаче показывают похожий результат.
Очень важно понимание о сбалансированности классов или об их пропорции.
Метрика ROC AUC, как и все остальные, не может быть единственной метрикой качества модели. Необходимо рассматривать комплекс метрик (ROC AUC + precision + recall + f1 score и т.д.)
Кросс-валидация позволяет настроить гиперпараметры для оптимального решения.

In [ ]:
results.sort_values(by = 'F1', ascending = False)

In [ ]:
results.sort_values(by = 'ROC AUC', ascending = False)